# Flights: Model Selection

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
sns.set()

In [0]:
# Thiner line width
# globally
from matplotlib import rcParams
rcParams['patch.linewidth'] = 0.0 #0.5

In [3]:
import sklearn
print(sklearn.__version__)

0.22.1


In [0]:
COLAB = True

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/My Drive/Colab Notebooks/flight_features.csv'
else:
    data_path = 'flight_features.csv'    

In [0]:
df = pd.read_csv(data_path, index_col=0)

Feature description

- flight         - Flight number
- dep            - Departure airport name
- dst            - Desctination airport name
- plane_id       - Plane id (number)

- duration       - Flight duration
- dep_diff       - Scheduled and factual departure time differences
- dst_diff       - Scheduled and factual arrival time differences

- dep_cnt        - Departure flight counts per day, per airport
- dst_cnt        - Arrival flight counts per day, per airport

- dep_month      - Departure month
- dep_day        - Departure day
- dep_weekday    - Departure day of the week
- dep_hour       - Departure hour 
- dep_ minute    - Departure minute

- dst_month      - Arrival month
- dst_day        - Arrival day
- dst_weekday    - Arrival day of the week
- dst_hour       - Arrival hour 
- dst_ minute    - Arrival minute


In [8]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139732 entries, 0 to 139731
Data columns (total 19 columns):
flight         139732 non-null int64
dep            139732 non-null object
dst            139732 non-null object
plane_id       139732 non-null int64
duration       139732 non-null float64
dep_diff       139732 non-null float64
dst_diff       139732 non-null float64
dep_cnt        139732 non-null int64
dst_cnt        139732 non-null int64
dep_month      139732 non-null int64
dep_day        139732 non-null int64
dep_weekday    139732 non-null int64
dep_hour       139732 non-null int64
dep_ minute    139732 non-null int64
dst_month      139732 non-null int64
dst_day        139732 non-null int64
dst_weekday    139732 non-null int64
dst_hour       139732 non-null int64
dst_ minute    139732 non-null int64
dtypes: float64(3), int64(14), object(2)
memory usage: 21.3+ MB


,flight,dep,dst,plane_id,duration,dep_diff,dst_diff,dep_cnt,dst_cnt,dep_month,dep_day,dep_weekday,dep_hour,dep_ minute,dst_month,dst_day,dst_weekday,dst_hour,dst_ minute
0,1,MOW,ARH,1,115.0,-17.0,-12.0,329,3,3,1,3,0,5,3,1,3,2,0
1,2,MOW,EGO,2,85.0,-5.0,25.0,329,2,3,1,3,0,5,3,1,3,1,30
2,3,MOW,AKX,3,145.0,-1.0,-8.0,329,1,3,1,3,0,15,3,1,3,2,40
3,4,LED,MOW,4,75.0,-9.0,4.0,20,331,3,1,3,0,20,3,1,3,1,35
4,5,MOW,KRR,5,135.0,-4.0,1.0,329,7,3,1,3,0,25,3,1,3,2,40


# Prepare data

In [9]:
# Data for model selection
X = df[['dep','dst','plane_id','duration', 'dep_cnt', 'dst_cnt', 
         'dep_month', 'dep_day', 'dep_weekday', 'dep_hour','dep_ minute', 
         'dst_month', 'dst_day', 'dst_weekday', 'dst_hour','dst_ minute']].copy()

y = df['dep_diff'].copy()
y = y.values.reshape(-1,1)
print("X.shape, y.shape: ", X.shape, y.shape)

X.shape, y.shape:  (139732, 16) (139732, 1)


In [0]:
# Break off validation set from training data
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            train_size=0.8, test_size=0.2,
                                                            random_state=0)



In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipe = Pipeline([
         ('std_scaler', StandardScaler())
    ])

In [12]:
# Select numerical columns
num_attribs = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]

print('Numerical attributes: ', num_attribs)

Numerical attributes:  ['plane_id', 'duration', 'dep_cnt', 'dst_cnt', 'dep_month', 'dep_day', 'dep_weekday', 'dep_hour', 'dep_ minute', 'dst_month', 'dst_day', 'dst_weekday', 'dst_hour', 'dst_ minute']


In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [0]:
cat_attribs = ['dep','dst']

pipe = ColumnTransformer([
        ("num", num_pipe, num_attribs),
        ("cat", OneHotEncoder(handle_unknown='ignore'), cat_attribs),
    ])
X_train_prep = pipe.fit_transform(X_train)

In [15]:
type(X_train_prep), X_train_prep.shape, X.shape

(scipy.sparse.csr.csr_matrix, (111785, 326), (139732, 16))

In [0]:
from sklearn.preprocessing import LabelEncoder
# Apply label encoder to 'departures' and 'arrivals' 
label_encoder = LabelEncoder()
X_train_label = X_train.copy()
X_train_label['dep'] = label_encoder.fit_transform(X_train_label['dep'])
X_train_label['dst'] = label_encoder.fit_transform(X_train_label['dst'])
X_train_label = num_pipe.fit_transform(X_train_label)

# Select and train a model 

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

In [0]:
def train_predict(model, x,y):
    """Train model with default parameters, predict on train set, 
    return MAE, RMSE"""
    model.fit(x,y)
    preds = model.predict(x)
    mae = mean_absolute_error(y,preds)
    rmse = np.sqrt(mean_squared_error(y,preds))
    return mae, rmse

In [19]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
mae, rmse = train_predict(lin_reg, X_train_prep, y_train)
print("Linear Regression MAE: ", mae, " RMSE: ",rmse)

Linear Regression MAE:  9.930176244895035  RMSE:  45.91879146924766


In [20]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=30, random_state=78) #n_estimators=100
mae, rmse = train_predict(forest_reg, X_train_prep, y_train)
print(" RandomForest MAE: ", mae, " RMSE: ",rmse)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


 RandomForest MAE:  4.308375601974028  RMSE:  19.53618220131576


In [21]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(#objective ='reg:linear', 
    objective = 'reg:squarederror',
    colsample_bytree = 0.3, learning_rate = 0.1,
    max_depth = 5, alpha = 10, n_estimators = 10)
mae, rmse = train_predict(xg_reg, X_train_prep, y_train)
print(" XGBRegressor MAE: ", mae, " RMSE: ",rmse)

 XGBRegressor MAE:  10.47566828823913  RMSE:  45.41268035888441


In [22]:
# Try pipeline on a few training examples
some_X = X_train.iloc[:10]
some_y = y_train[:10]
some_X_prep = pipe.transform(some_X)

# Predictions and actual
list(zip(xg_reg.predict(some_X_prep), some_y))

[(-2.9029346, array([-7.])),
 (-2.0886698, array([-8.])),
 (-2.9029346, array([-12.])),
 (-3.0241313, array([-12.])),
 (3.9720008, array([44.])),
 (-1.8526938, array([-5.])),
 (-0.05630213, array([-2.])),
 (-2.4686422, array([-8.])),
 (-2.0886698, array([5.])),
 (-2.0886698, array([-11.]))]

In [0]:
def display_cv_scores(neg_scores):
    scores = np.sqrt(-neg_scores)
    print(pd.Series(scores).describe())

In [24]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=78)
scores = cross_val_score(tree_reg, X_train_prep, y_train,
                         scoring="neg_mean_squared_error", cv=10)
display_cv_scores(scores)

count    10.000000
mean     72.886223
std      10.957808
min      53.949633
25%      64.453365
50%      76.603553
75%      80.149862
max      84.586535
dtype: float64


In [25]:
scores = cross_val_score(lin_reg, X_train_prep, y_train, scoring="neg_mean_squared_error", cv=10)
display_cv_scores(scores)

count    10.000000
mean     44.506893
std      12.522558
min      30.988435
25%      34.781011
50%      42.670214
75%      46.596757
max      70.132700
dtype: float64


In [26]:
scores = cross_val_score(xg_reg, X_train_prep, y_train, scoring="neg_mean_squared_error", cv=10)
display_cv_scores(scores)

count    10.000000
mean     44.600265
std      12.483137
min      31.230889
25%      34.842000
50%      42.944498
75%      46.688189
max      70.131156
dtype: float64


In [0]:
from sklearn.model_selection import RandomizedSearchCV

def best_model_params(model, X_train_prep, y_train, param_distributions, n_iter=10, cv=2):
    """Use RandomizedSearchCV to find best model and params.
    Return: best model, best params, MAE and RMSE for train data.
    """
    rnd_search = RandomizedSearchCV(model, param_distributions=param_distributions,
                                n_iter=n_iter, cv=cv, scoring='neg_mean_squared_error', random_state=78)
    
    rnd_search.fit(X_train_prep, y_train)
    best_model = rnd_search.best_estimator_
    best_params = rnd_search.best_params_
    
    preds = best_model.predict(X_train_prep)
    mae = mean_absolute_error(y_train,preds)
    rmse = np.sqrt(mean_squared_error(y_train,preds))
    
    return best_model, best_params, mae, rmse

In [0]:
def display_feature_importance(rf_model):
    feature_importances = rf_model.feature_importances_
    importance_df = pd.DataFrame(feature_importances, index=X_train.columns, 
                      columns=["Importance"])
    print(importance_df.sort_values(by=['Importance'], ascending=False))
   

In [29]:
y_train = y_train.reshape(-1)
y_train.shape

(111785,)

In [30]:
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]
forest_reg = RandomForestRegressor(random_state=78)
rf1_best_model, rf1_best_params, rf1_mae, rf1_rmse = best_model_params(forest_reg, X_train_prep, y_train, param_grid)
print("RandomForest 1 (onehot)  MAE: ", rf1_mae, " RMSE: ", rf1_rmse)

rf2_best_model, rf2_best_params, rf2_mae, rf2_rmse = best_model_params(forest_reg, X_train_label, y_train, param_grid)
print("RandomForest 2 (label)  MAE: ", rf2_mae, " RMSE: ", rf2_rmse)

RandomForest 1 (onehot)  MAE:  3.9830516914911067  RMSE:  17.871643449217864
RandomForest 2 (label)  MAE:  4.0630603390437  RMSE:  17.983076620358656


In [31]:
display_feature_importance(rf2_best_model)

             Importance
plane_id       0.158807
dep_day        0.092219
dst_cnt        0.080570
duration       0.074990
dep            0.071132
dst_day        0.070447
dep_hour       0.062066
dst_hour       0.061999
dst_ minute    0.060725
dep_ minute    0.058116
dep_cnt        0.048382
dep_weekday    0.048243
dst_weekday    0.042495
dst_month      0.032495
dep_month      0.026082
dst            0.011231


In [32]:
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }
rf3_best_model, rf3_best_params, rf3_mae, rf3_rmse = best_model_params(
    forest_reg, X_train_prep, y_train, param_distribs)
print("RandomForest 3 (onehot)  MAE: ", rf3_mae, " RMSE: ", rf3_rmse)

rf4_best_model, rf4_best_params, rf4_mae, rf4_rmse = best_model_params(
    forest_reg, X_train_label, y_train, param_distribs)
print("RandomForest 4 (label)  MAE: ", rf4_mae, " RMSE: ", rf4_rmse)

RandomForest 3 (onehot)  MAE:  3.7723902547577866  RMSE:  17.258753706237865
RandomForest 4 (label)  MAE:  3.8038275125514702  RMSE:  17.311345169070698


In [33]:
display_feature_importance(rf4_best_model)

             Importance
plane_id       0.142857
duration       0.088481
dst_day        0.079808
dep            0.079373
dep_day        0.075968
dst_cnt        0.070955
dep_hour       0.070711
dst_hour       0.067779
dst_ minute    0.063558
dep_ minute    0.063208
dep_cnt        0.055583
dep_weekday    0.040285
dst_weekday    0.036241
dep_month      0.026382
dst_month      0.023380
dst            0.015433


In [34]:
from sklearn.linear_model import ElasticNet
l1_space = np.linspace(0,1,30)
param_grid = {'l1_ratio':l1_space}
elastic_net = ElasticNet()
lr_best_model, lr_best_params, lr_mae, lr_rmse = best_model_params(elastic_net, X_train_prep, y_train, param_grid)
print("LR MAE: ", lr_mae, " RMSE: ", lr_rmse)

LR MAE:  9.857046634395187  RMSE:  46.11552500546665


In [35]:
import xgboost as xgb
import scipy.stats as st

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}

xgb_reg = xgb.XGBRegressor(objective = 'reg:squarederror', nthreads=-1)  
#xgb_reg = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4) 
xgb_best_model, xgb_best_params, xgb_mae, xgb_rmse = best_model_params(xgb_reg, X_train_prep, y_train, params)
print("XGBRegressor MAE: ", xgb_mae, " RMSE: ", xgb_rmse)

XGBRegressor MAE:  9.483694824780462  RMSE:  44.223646790584475


In [0]:
X_test_prepared = pipe.transform(X_test)

In [0]:
def predict(model):
  preds = model.predict(X_test_prepared)
  mae = mean_absolute_error(y_test,preds)
  rmse = np.sqrt(mean_squared_error(y_test,preds))
  return mae, rmse

In [38]:
xgb_mae, xgb_rmse = predict(xgb_best_model)
print("XGB MAE: ", xgb_mae," RMSE: ", xgb_rmse)


XGB MAE:  10.684400214025423  RMSE:  58.9953634826303


In [39]:
rf3_mae, rf3_rmse = predict(rf3_best_model)
print("RF3 MAE: ", rf3_mae," RMSE: ", rf3_rmse)

RF3 MAE:  10.824560184315825  RMSE:  59.31106302905509


In [40]:
rf1_mae, rf1_rmse = predict(rf1_best_model)
print("RF1 MAE: ", rf1_mae," RMSE: ", rf1_rmse)

RF1 MAE:  11.115959971851481  RMSE:  59.741441568899816


In [41]:
lr_mae, lr_rmse = predict(lr_best_model)
print("LR MAE: ", lr_mae," LR: ", lr_rmse)

LR MAE:  10.606749575439132  LR:  59.272504959592666
